In [1]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import warnings
import seaborn as sns

In [2]:
columns = ['CVE']
metasploit_df = pd.read_csv('metasploit.txt', header=None, names=columns)
nuclei_df = pd.read_csv('nuclei.txt', header=None, names=columns)


In [3]:
CISA_df = pd.read_csv('known_exploited_vulnerabilities.csv')
CISA_df = CISA_df.rename(columns={"cveID": "CVE"})

In [4]:
epss_df = pd.read_csv('epss_scores-current.csv', skiprows=1)
epss_df = epss_df.rename(columns={"cve": "CVE"})
epss_df_all = epss_df
epss_df = epss_df[epss_df.epss > .95]

In [5]:
epss_cves = epss_df['CVE']
metasploit_cves = metasploit_df['CVE']
nuclei_cves = nuclei_df['CVE']
CISA_cves = CISA_df['CVE']

In [6]:
CVE_list = pd.merge(CISA_cves, metasploit_cves, how='outer', left_on='CVE', right_on='CVE')
CVE_list = pd.merge(CVE_list, nuclei_cves, how='outer', left_on='CVE', right_on='CVE')
CVE_list = pd.merge(CVE_list, epss_cves, how='outer', left_on='CVE', right_on='CVE')
CVE_list = CVE_list.drop_duplicates()


In [7]:
row_accumulator = []
for filename in glob.glob('nvd.jsonl'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data:
            cve = entry['cve']['id']
            try:
                assigner = entry['cve']['sourceIdentifier']
            except KeyError:
                assigner = 'Missing_Data'
            try:
                published_date = entry['cve']['published']
            except KeyError:
                published_date = 'Missing_Data'
            try:
                attack_vector = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['attackVector']
            except KeyError:
                attack_vector = 'Missing_Data'
            try:
                attack_complexity = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['attackComplexity']
            except KeyError:
                attack_complexity = 'Missing_Data'
            try:
                privileges_required = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['privilegesRequired']
            except KeyError:
                privileges_required = 'Missing_Data'
            try:
                user_interaction = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['userInteraction']
            except KeyError:
                user_interaction = 'Missing_Data'
            try:
                scope = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['scope']
            except KeyError:
                scope = 'Missing_Data'
            try:
                confidentiality_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['confidentialityImpact']
            except KeyError:
                confidentiality_impact = 'Missing_Data'
            try:
                integrity_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['integrityImpact']
            except KeyError:
                integrity_impact = 'Missing_Data'
            try:
                availability_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['availabilityImpact']
            except KeyError:
                availability_impact = 'Missing_Data'
            try:
                base_score = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']
            except KeyError:
                base_score = '0.0'
            try:
                base_severity = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseSeverity']
            except KeyError:
                base_severity = 'Missing_Data'
            try:
                exploitability_score = entry['cve']['metrics']['cvssMetricV31'][0]['exploitabilityScore']
            except KeyError:
                exploitability_score = 'Missing_Data'
            try:
                impact_score = entry['cve']['metrics']['cvssMetricV31'][0]['impactScore']
            except KeyError:
                impact_score = 'Missing_Data'
            try:
                cwe = entry['cve']['weaknesses'][0]['description'][0]['value']
            except KeyError:
                cwe = 'Missing_Data'
            try:
                description = entry['cve']['descriptions'][0]['value']
            except IndexError:
                description = ''
            new_row = {
                'CVE': cve,
                'Published': published_date,
                'CVSS Score': base_score,
                'Description': description
            }
            if not description.startswith('** REJECT **'): # disputed, rejected and other non issues start with '**'
                row_accumulator.append(new_row)
        nvd = pd.DataFrame(row_accumulator)
        
nvd['Published'] = pd.to_datetime(nvd['Published'])
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)

In [8]:
patchthisapp_df = pd.merge(CVE_list, nvd, how='inner', left_on='CVE', right_on='CVE')
patchthisapp_df = pd.merge(patchthisapp_df, epss_df_all, how='inner', left_on='CVE', right_on='CVE')
patchthisapp_df = patchthisapp_df[['CVE', 'CVSS Score', 'epss', 'Description', 'Published']]
patchthisapp_df = patchthisapp_df.rename(columns={"epss": "EPSS"})
patchthisapp_df.to_csv('data/data.csv', index=False)